# Control de Partícula en Tiempo Mínimo (Con Fricción)
---
El problema está planteado en Bertsekas Vol1 pag 144.

Se modifica el problema de control de la partícula, ahora esta se mueve con fricción en una recta, con condición inicial conocida $x(0)$ y $\dot{x}(0)$. Nuevamente, controlamos la partícula mediante una fuerza $u$ tal que $|u|\leq 1$.

Tenemos como objetivo llevar la partícula al origen en tiempo mínimo. O sea que $x(T)=0$ y $\dot{x}(T)=0$ con:

$$ \min_u \int_0^T 1 dt $$

## Solución mediante Principio del Mínimo de Pontryagin

En el problema con fricción, al igual que en el problema original, el principio del mínimo resulta en:
* $u^*=1$ si $p_2<0$
* $u^*=-1$ si $p_2>0$

En el problema sin fricción, la solucion de $p_2(t)$ a partir de las ecuaciones adjuntas era simplemente: $p_2(t)=at+b$.

En el problema actual, la EDO adjunta para $p_2(t)$ es un poco más complicada:

$$\dot{p}_2 = -a -bp_2$$

La solucion es: $p_2(t) = c e^{-bt}-a/b$. De esta ecuacion podemos concluir nuevamente que habrá a lo sumo un cambio de signo de $p_2$ y por lo tanto a lo sumo un switcheo.


El principio del mínimo nos indica nuevamente que van a haber cuatro alternativas de control dependiendo de cuál sea la condición inicial y que el control es del tipo Bang-Bang.

1. Switcheo de Fuerza:      $u^*=1$ a $u^*=-1$
2. Switcheo de Fuerza:      $u^*=-1$ a $u^*=1$
3. Mantengo siempre Fuerza: $u^*=1$
4. Mantengo siempre Fuerza: $u^*=-1$


De la política de switcheos, concluímos que debemos resolver la dinámica tanto para $u=1$ como para $u=-1$ y las trayectorias en el espacio de fases ($[x;\dot{x}]$) obtenidas se debían concatenar para obtener las trayectorias óptimas.

<img src="Part_Friccion.png" width="400">


## Solución Numérica usando JuMP

In [ ]:
using JuMP, Ipopt

In [ ]:
# Codigo basado en ejemplo "Rocket Control" (https://www.juliaopt.org/notebooks/JuMP-Rocket.html)

# Creo Modelo en JuMP, uso Ipopt como solver
mod = Model(with_optimizer(Ipopt.Optimizer,print_level=0))

# Ejemplo de Control de una particula de masa=1 mediante fuerzas |u|<=1
# de manera de llegar a x=0 con v=0 en tiempo minimo. Con Friccion.

#Mostrar los puntos de arranque (-1,-1); (-2,2) y (1,2)

x_0 = 1    # Posicion Inicial
v_0 = 2    # Velocidad Inicial

n = 50                                     # Numero de Pasos Temporales
@variable(mod, Δt ≥ 0, start = 1/n)        # Intervalo de Tiempo
@NLexpression(mod,t_f, Δt*n)               # Tiempo Final

# Variables Dinamicas
@variable(mod, x[0:n] )          # Posicion
@variable(mod, v[0:n] )          # Velocidad

# Control: Empuje
@variable(mod, -1 ≤ u[0:n] ≤ 1)

# Objetivo: Minimizar tiempo hasta llegar a x=0 con v=0
@NLobjective(mod, Min, t_f)

# Condiciones Iniciales
@constraint(mod, x[0] == x_0)
@constraint(mod, v[0] == v_0)

# Condiciones Finales
@constraint(mod, x[n] == 0)
@constraint(mod, v[n] == 0)


# Dinamica
for j in 1:n
    # x' = v
    # Regla del Trapecio
    @NLconstraint(mod,
        x[j] == x[j-1] + 0.5*Δt*(v[j]+v[j-1]))

    # v' = u - v
    # Regla del Trapecio
    @NLconstraint(mod,
    
    v[j] == v[j-1] + 0.5*Δt*(u[j]-v[j]+u[j-1]-v[j-1]))
end

# Punto de Arranque Para Solver
for k in 0:n
    set_start_value(x[k], 1)
    set_start_value(v[k], 0)
    set_start_value(u[k], 1)
end

# Imprimo el Problema de Optimizacion (mod)
#print(mod)
println("")

# Resolvemos control y variables de estado
println("Solving...")
status = optimize!(mod)

# Mostramos resultados
println("Estado del Solver: ", termination_status(mod))
println("Tiempo de Llegada al Origen: ", objective_value(mod))

## Presentación gráfica de soluciones numéricas

In [ ]:
using Gadfly

In [ ]:
tiempo = collect((0:n)*value(Δt))

x_plot = plot(x=tiempo,y=collect(value.(x)), Geom.point,
Guide.xlabel("Tiempo"), Guide.ylabel("Posicion"))

v_plot = plot(x=tiempo,y=collect(value.(v)), Geom.point,
Guide.xlabel("Tiempo"), Guide.ylabel("Velocidad"))

f_plot = plot(x=collect(value.(x)),y=collect(value.(v)), Geom.point,
            Guide.xlabel("Posicion"), Guide.ylabel("Velocidad"))

u_plot = plot(x=tiempo,y=collect(value.(u)), Geom.point,
            Guide.xlabel("Tiempo"), Guide.ylabel("Fuerza Control (u)"))

draw(SVG(6inch, 6inch), vstack(hstack( x_plot,v_plot),
                        hstack(f_plot,u_plot)))